In [186]:
%matplotlib inline
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt
import uqer
from PyFin.api import *
from uqer import DataAPI as api
from QuantLib import *

plt.style.use('fivethirtyeight')
uqer.Client(token='f1b9bea1d0b4e489c5ab9b69c3e2326a1bee6057af858067dbd1546453f428b2')

16937@wmcloud.com 账号登录成功


In [187]:
start_date = '2017-04-01'
end_date = '2017-08-01'

dates = makeSchedule(start_date, end_date, '1m', 'china.sse', )

mkt_start_date = advanceDateByCalendar('china.sse', start_date, '-30b').strftime('%Y-%m-%d')
contract = 'ru1709'

In [188]:
def get_contracts(contract, start_date, end_date):
    data = api.MktFutdGet(ticker=contract, beginDate=start_date, endDate=end_date, field='tradeDate,closePrice')
    data['tradeDate'] = pd.to_datetime(data.tradeDate)
    data.set_index('tradeDate', inplace=True)
    data['closePrice'] = data['closePrice'].astype(float)
    data['ret'] = data.closePrice.diff() / data.closePrice.shift(1)
    data['vol'] = data['ret'].rolling(window=22).std() * np.sqrt(249)
    return data

In [441]:
def get_contracts_bar(constract, start_date, end_date, unit=60):
    start_date = dt.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = dt.datetime.strptime(end_date, '%Y-%m-%d')
    
    datas = []
    while start_date < end_date:
        this_end_date = advanceDateByCalendar('china.sse', start_date, '10b')
        df = api.MktFutureBarHistDateRangeGet(instrumentID=contract,
                                              startDate=start_date.strftime('%Y%m%d'),
                                              endDate=this_end_date.strftime('%Y%m%d'),
                                              unit=unit,
                                              field='dataDate,barTime,closePrice')
        datas.append(df)
        start_date = this_end_date
    data = pd.concat(datas)
    data = data[(data.barTime >= '09:00') & (data.barTime <= '16:00')]
    data['tradeDate'] = pd.to_datetime(data.dataDate + 'T' +  data.barTime)
    data['closePrice'] = data['closePrice'].astype(float)
    data['ret'] = data.closePrice.diff() / data.closePrice.shift(1)
    data['vol'] = data['ret'].rolling(window=int(22 * 4 * 60 / unit)).std() * np.sqrt(249) * np.sqrt(4 * 60 / unit)
    data.set_index('tradeDate', inplace=True)
    return data[['closePrice', 'ret', 'vol']]

In [442]:
mkt_data = get_contracts_bar(contract, mkt_start_date, end_date)

In [443]:
strangle_width = 400.
lower_protect_area = 1000.
upper_protect_area = 800.
volatility = 0.42
risk_free_rate = 0.
dividend_rate = 0.

In [444]:
mkt_data.tail()

,closePrice,ret,vol
tradeDate,,,
2017-12-11 15:00:00,13550.0,-0.001474,0.329665
2017-12-12 10:00:00,13680.0,0.009594,0.331387
2017-12-12 11:15:00,13650.0,-0.002193,0.331427
2017-12-12 14:15:00,13590.0,-0.004396,0.331701
2017-12-12 15:00:00,13670.0,0.005887,0.332368


# Dynamical Hedging
------------

In [445]:
def cal_naive_delta(spot, pp1, pp2, cp1, cp2):
    
    if spot <= pp1.strike():
        return 0.
    elif spot <= pp2.strike():
        return -1.
    elif spot <= cp1.strike():
        return 0.
    elif spot <= cp2.strike():
        return 1.
    else:
        return 0.

def hedging_contract(start_date, maturity_date, mkt_data, premium=None, naive_hedge=False):
    spot_price = mkt_data[mkt_data.index < start_date.ISO()]['closePrice']
    
    if hasattr(spot_price, '__iter__'):
        spot_price = spot_price[-1]
    
    lower_strike = spot_price
    lower_bound = lower_strike - lower_protect_area
    upper_strike = lower_strike + strangle_width
    upper_bound = upper_strike + upper_protect_area
    
    day_count = Actual365Fixed()
    calendar = China(China.SSE)
    
    calculation_date = start_date
    Settings.instance().evaluationDate = calculation_date

    spot_handle = RelinkableQuoteHandle(SimpleQuote(spot_price))
    vol_handle =  RelinkableQuoteHandle(SimpleQuote(volatility))

    flat_ts = YieldTermStructureHandle(FlatForward(calculation_date, risk_free_rate, day_count))
    dividend_yield = YieldTermStructureHandle(FlatForward(calculation_date, dividend_rate, day_count))
    flat_vol_ts = BlackVolTermStructureHandle(BlackConstantVol(calculation_date, calendar, vol_handle, day_count))

    bsm_process = BlackScholesMertonProcess(spot_handle, dividend_yield, flat_ts, flat_vol_ts)
    
    put_payoff1 = PlainVanillaPayoff(Option.Put, lower_strike)
    put_payoff2 = PlainVanillaPayoff(Option.Put, lower_bound)
    call_payoff1 = PlainVanillaPayoff(Option.Call, upper_strike)
    call_payoff2 = PlainVanillaPayoff(Option.Call, upper_bound)

    exercise = EuropeanExercise(maturity_date)
    put_option1 = VanillaOption(put_payoff1, exercise)
    put_option2 = VanillaOption(put_payoff2, exercise)
    call_option1 = VanillaOption(call_payoff1, exercise)
    call_option2 = VanillaOption(call_payoff2, exercise)

    engine = AnalyticEuropeanEngine(bsm_process)
    put_option1.setPricingEngine(engine)
    put_option2.setPricingEngine(engine)
    call_option1.setPricingEngine(engine)
    call_option2.setPricingEngine(engine)
    
    if not premium:
        premium = put_option1.NPV() - put_option2.NPV() + call_option1.NPV() - call_option2.NPV()
    
    price_series = mkt_data.loc[start_date.ISO(): maturity_date.ISO()]
    previous_delta = 0.
    previous_price = 0.
    trading_pnl = 0.
    for p in price_series.iterrows():
        date = Date.from_date(p[0])
        price = p[1]['closePrice']
        vol = p[1]['vol']
        if date != maturity_date:
            Settings.instance().evaluationDate = date
        spot_handle.linkTo(SimpleQuote(price))
        vol_handle.linkTo(SimpleQuote(vol))
        bsm_delta = put_option1.delta() - put_option2.delta() + call_option1.delta() - call_option2.delta()
        
        if naive_hedge:
            delta = cal_naive_delta(price, put_payoff1, put_payoff2, call_payoff1, call_payoff2)
        else:
            delta = bsm_delta
        
        trading_pnl += (price - previous_price) * previous_delta
        previous_price = price
        previous_delta = delta
        
    final_price = mkt_data.loc[maturity_date.ISO()]['closePrice']
    if hasattr(final_price, '__iter__'):
        final_price = final_price[-1]
    final_payoff = put_payoff1(final_price) - put_payoff2(final_price) + call_payoff1(final_price) - call_payoff2(final_price)
    
    pnl_final = -final_payoff + premium + trading_pnl
    
    return spot_price, final_price, vol, final_payoff, premium, trading_pnl, pnl_final

In [446]:
maturity_date = Date.from_date(advanceDateByCalendar('china.sse', dates[2], '-1b'))
start_date = Date.from_date(dates[1])

hedging_contract(start_date, maturity_date, mkt_data, 646.)

(16545.0,
 13495.0,
 0.38444123153256793,
 1000.0,
 646.0,
 357.3622073541228,
 3.3622073541228019)

# Historical Simulation
-----------------

## 日间对冲
------------

In [447]:
strangle_width = 400.
lower_protect_area = 1000.
upper_protect_area = 800.
volatility = 0.42
risk_free_rate = 0.
dividend_rate = 0.
premium = 646.

In [448]:
contract_definition = {
    'ru1309': {
        'start_date': '2013-04-01',
        'end_date': '2013-08-01'
    },
    'ru1401': {
        'start_date': '2013-08-01',
        'end_date': '2013-12-01'
    },
    'ru1409': {
        'start_date': '2014-04-01',
        'end_date': '2014-08-01'
    },
     'ru1501': {
        'start_date': '2014-08-01',
        'end_date': '2014-12-01'
    },
    'ru1509': {
        'start_date': '2015-04-01',
        'end_date': '2015-08-01'
    },
    'ru1601': {
        'start_date': '2015-08-01',
        'end_date': '2015-12-01'
    },
    'ru1609': {
        'start_date': '2016-04-01',
        'end_date': '2016-08-01'
    },
    'ru1701': {
        'start_date': '2016-08-01',
        'end_date': '2016-12-01'
    },
    'ru1709': {
        'start_date': '2017-04-01',
        'end_date': '2017-08-01'
    },
    'ru1801': {
        'start_date': '2017-08-01',
        'end_date': '2017-12-01'
    },
}

In [449]:
%%time

cols = ['合约', '到期日', '初始价', '收盘价', '实现波动率', '赔付', '期权费', '交易损益', '对冲损益', 'delta 1 对冲损益']
df = pd.DataFrame(columns=cols)

for contract in contract_definition:
    start_date = contract_definition[contract]['start_date']
    end_date = contract_definition[contract]['end_date']
    dates = makeSchedule(start_date, end_date, '1m', 'china.sse')
    mkt_start_date = advanceDateByCalendar('china.sse', start_date, '-30b').strftime('%Y-%m-%d')
    mkt_data = get_contracts(contract, mkt_start_date, end_date)
    
    for i, s_date in enumerate(dates[:-1]):
        maturity_date = Date.from_date(advanceDateByCalendar('china.sse', dates[i+1], '-1b'))
        start_date = Date.from_date(s_date)

        hedge = hedging_contract(start_date, maturity_date, mkt_data, premium)
        delta1_hedge = hedging_contract(start_date, maturity_date, mkt_data, premium, True)
        py_dt = dt.datetime(maturity_date.year(), maturity_date.month(), maturity_date.dayOfMonth())
        df = df.append(dict(zip(cols, (contract, py_dt) + hedge + (delta1_hedge[-1],))), ignore_index=True)

Wall time: 1.27 s


In [450]:
df['不对冲损益'] = -df['赔付'] + df['期权费']

In [451]:
df.head()

,合约,到期日,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,delta 1 对冲损益,不对冲损益
0,ru1309,2013-04-26,22165.0,19080.0,0.323892,1000.0,646.0,330.927109,-23.072891,-354.0,-354.0
1,ru1309,2013-05-31,19080.0,18740.0,0.312908,340.0,646.0,-69.120001,236.879999,1411.0,306.0
2,ru1309,2013-06-28,18740.0,17005.0,0.273979,1000.0,646.0,229.855685,-124.144315,-689.0,-354.0
3,ru1309,2013-07-31,17005.0,16865.0,0.295579,140.0,646.0,-102.765998,403.234002,111.0,506.0
4,ru1401,2013-08-30,17915.0,20000.0,0.246227,800.0,646.0,27.845411,-126.154589,-14.0,-154.0


In [452]:
df.groupby(df['到期日'].dt.year).mean()

,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,delta 1 对冲损益,不对冲损益
到期日,,,,,,,,,
2013,19290.000,18768.750,0.261487,539.375,646.0,30.506474,137.131474,-83.375,106.625
2014,14363.750,13788.125,0.229827,690.625,646.0,169.106949,124.481949,101.625,-44.625
2015,12620.000,12336.250,0.264896,545.625,646.0,65.793237,166.168237,67.875,100.375
2016,12217.500,12777.500,0.317360,658.125,646.0,47.432287,35.307287,109.125,-12.125
2017,14445.625,13744.375,0.333330,656.250,646.0,22.757294,12.507294,-265.250,-10.250


In [453]:
df.std()

初始价             2872.994950
收盘价             2747.603105
实现波动率              0.074985
赔付               342.140322
期权费                0.000000
交易损益             201.898770
对冲损益             220.873920
delta 1 对冲损益     626.309909
不对冲损益            342.140322
dtype: float64

In [454]:
df.mean()

初始价             14587.375000
收盘价             14283.000000
实现波动率               0.281380
赔付                618.000000
期权费               646.000000
交易损益               67.119248
对冲损益               95.119248
delta 1 对冲损益      -14.000000
不对冲损益              28.000000
dtype: float64

## 日内对冲
---------------

In [455]:
%%time

cols = ['合约', '到期日', '初始价', '收盘价', '实现波动率', '赔付', '期权费', '交易损益', '对冲损益', 'delta 1 对冲损益']
df2 = pd.DataFrame(columns=cols)

for contract in contract_definition:
    start_date = contract_definition[contract]['start_date']
    end_date = contract_definition[contract]['end_date']
    dates = makeSchedule(start_date, end_date, '1m', 'china.sse')
    mkt_start_date = advanceDateByCalendar('china.sse', start_date, '-30b').strftime('%Y-%m-%d')
    mkt_data = get_contracts_bar(contract, mkt_start_date, end_date, unit=60)
    
    for i, s_date in enumerate(dates[:-1]):
        maturity_date = Date.from_date(advanceDateByCalendar('china.sse', dates[i+1], '-1b'))
        start_date = Date.from_date(s_date)

        hedge = hedging_contract(start_date, maturity_date, mkt_data, premium)
        delta1_hedge = hedging_contract(start_date, maturity_date, mkt_data, premium, True)
        py_dt = dt.datetime(maturity_date.year(), maturity_date.month(), maturity_date.dayOfMonth())
        df2 = df2.append(dict(zip(cols, (contract, py_dt) + hedge + (delta1_hedge[-1],))), ignore_index=True)

Wall time: 16.5 s


In [456]:
df2['不对冲损益'] = -df['赔付'] + df['期权费']

In [457]:
df2.head()

,合约,到期日,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,delta 1 对冲损益,不对冲损益
0,ru1309,2013-04-26,22165.0,19080.0,0.363292,1000.0,646.0,389.349380,35.349380,-354.0,-354.0
1,ru1309,2013-05-31,19080.0,18740.0,0.242909,340.0,646.0,-136.945424,169.054576,1471.0,306.0
2,ru1309,2013-06-28,18740.0,17005.0,0.238767,1000.0,646.0,269.465020,-84.534980,-729.0,-354.0
3,ru1309,2013-07-31,17005.0,16865.0,0.247096,140.0,646.0,-210.046889,295.953111,-129.0,506.0
4,ru1401,2013-08-30,17915.0,20000.0,0.221684,800.0,646.0,152.554679,-1.445321,416.0,-154.0


In [458]:
df2.groupby(df2['到期日'].dt.year).mean()

,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,delta 1 对冲损益,不对冲损益
到期日,,,,,,,,,
2013,19290.000,18768.750,0.232745,539.375,646.0,26.814663,133.439663,-47.750,106.625
2014,14386.875,13807.500,0.227062,697.500,646.0,137.019454,85.519454,-17.750,-44.625
2015,12620.000,12336.250,0.281842,545.625,646.0,35.866448,136.241448,37.875,100.375
2016,12217.500,12777.500,0.323096,658.125,646.0,25.704331,13.579331,376.000,-12.125
2017,14445.625,13744.375,0.332807,656.250,646.0,23.985122,13.735122,-210.875,-10.250


In [459]:
df2.std()

初始价             2873.211328
收盘价             2747.942514
实现波动率              0.073859
赔付               343.001808
期权费                0.000000
交易损益             215.070090
对冲损益             213.432214
delta 1 对冲损益     751.711892
不对冲损益            342.140322
dtype: float64

In [460]:
df2.mean()

初始价             14592.000000
收盘价             14286.875000
实现波动率               0.279510
赔付                619.375000
期权费               646.000000
交易损益               49.878003
对冲损益               76.503003
delta 1 对冲损益       27.500000
不对冲损益              28.000000
dtype: float64

## 比较
-------------

In [461]:
s = df['不对冲损益'] - df2['不对冲损益']

In [462]:
df['diff'] = s

In [463]:
df[df['diff'] != 0]

,合约,到期日,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,delta 1 对冲损益,不对冲损益,diff


In [464]:
df2[df['diff'] != 0]

,合约,到期日,初始价,收盘价,实现波动率,赔付,期权费,交易损益,对冲损益,delta 1 对冲损益,不对冲损益
